In [1]:
import numpy as np
import matplotlib.pylab as plt
from numba import cuda
import math

In [38]:
#creates coordinate arrays of sources and evaluation points

npoints = 400 #resolution of modelling, number of points within the range
nsources = 4
sigma = .1

plot_gridx = np.mgrid[0:3:npoints * 1j, 0:3:npoints * 1j]
plot_gridy = np.mgrid[-1.5:1.5:npoints * 1j, -1.5:1.5:npoints * 1j]

#creating 3D array of coordinates for the three different plotting planes
targets = np.vstack((plot_gridx[0].ravel(),
                        plot_gridy[1].ravel(),
                        np.zeros(plot_grid[0].size))).T

rand = np.random.RandomState(0)


# We are picking random sources
sources = (rand.rand(nsources, 3)*2 - 1) #creates coordinates of sources
sources[:,0] = -1.2

In [39]:
print(plot_grid[0])
print()
print(plot_grid[0].ravel())
print()
print('\n','Targets:',targets.shape,'\n',targets)
print('\n','Sources:','\n',sources)

[[-1.         -1.         -1.         ... -1.         -1.
  -1.        ]
 [-0.99498747 -0.99498747 -0.99498747 ... -0.99498747 -0.99498747
  -0.99498747]
 [-0.98997494 -0.98997494 -0.98997494 ... -0.98997494 -0.98997494
  -0.98997494]
 ...
 [ 0.98997494  0.98997494  0.98997494 ...  0.98997494  0.98997494
   0.98997494]
 [ 0.99498747  0.99498747  0.99498747 ...  0.99498747  0.99498747
   0.99498747]
 [ 1.          1.          1.         ...  1.          1.
   1.        ]]

[-1. -1. -1. ...  1.  1.  1.]


 Targets: (160000, 3) 
 [[ 0.         -1.5         0.        ]
 [ 0.         -1.4924812   0.        ]
 [ 0.         -1.48496241  0.        ]
 ...
 [ 3.          1.48496241  0.        ]
 [ 3.          1.4924812   0.        ]
 [ 3.          1.5         0.        ]]

 Sources: 
 [[-1.2         0.43037873  0.20552675]
 [-1.2        -0.1526904   0.29178823]
 [-1.2         0.783546    0.92732552]
 [-1.2         0.58345008  0.05778984]]


In [10]:
SX = 16
SY = nsources

@cuda.jit
def rbf_evaluation_cuda(sources, targets, result, k):
    local_result = cuda.shared.array((SX, nsources), numba.float32)
    local_targets = cuda.shared.array((SX, 3), numba.float32)
    local_sources = cuda.shared.array((SY, 3), numba.float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    px, py = cuda.grid(2)
    
    if px >= targets.shape[0]:
        return

    
    # At first we are loading all the targets into the shared memory
    # We use only the first column of threads to do this.
    if ty == 0:
        for index in range(3):
            local_targets[tx, index] = targets[px, index]
    
    
    # We are now loading all the sources and weights.
    # We only require the first row of threads to do this.
    if tx == 0:
        for index in range(3):
            local_sources[ty, index] = sources[py, index]
        
        
    # Let us now sync all threads
    cuda.syncthreads()
    
    
    # Now compute the interactions
    squared_diff = numba.float32(0)
    
    for index in range(3):
        squared_diff += (local_targets[tx, index] - local_sources[ty, index])**2
    diff = np.sqrt(squared_diff)
    local_result[tx, ty] = math.cos(numba.float32(k) * diff) / (numba.float32(4) * numba.float32(np.pi) * diff)

   
    cuda.syncthreads() #re-sync threads
    
    # Now sum up all the local results
    if ty == 0:
        res = numba.float32(0)
        for index in range(nsources):
            res += local_result[tx, index]
        result[px] = res    

In [46]:
plt.rcParams["font.size"] = 16

result = np.zeros(len(targets), dtype=np.float64)

def visualize(result, npoints):
    """A helper function for visualization"""
    
    result_xy = result[: npoints * npoints].reshape(npoints, npoints).T

    fig = plt.figure(figsize=(20, 20))    

    ax = fig.add_subplot(1, 1, 1)   
    im = ax.imshow(result_xy, extent=[0, 1, 0, 1], origin='lower')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

In [47]:
nblocks = (targets.shape[0] + SX - 1) // SX
result = np.zeros(len(targets), dtype=np.float32)
rbf_evaluation_cuda[(nblocks, 1), (SX, SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), result)

NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
dlopen(libnvvm.dylib, 0x0006): tried: '/Users/harvey/opt/anaconda3/lib/libnvvm.dylib' (no such file), '/Users/harvey/opt/anaconda3/lib/libnvvm.dylib' (no such file), '/Users/harvey/opt/anaconda3/lib/python3.8/lib-dynload/../../libnvvm.dylib' (no such file), '/Users/harvey/opt/anaconda3/lib/libnvvm.dylib' (no such file), '/Users/harvey/opt/anaconda3/bin/../lib/libnvvm.dylib' (no such file), 'libnvvm.dylib' (no such file), '/usr/local/lib/libnvvm.dylib' (no such file), '/usr/lib/libnvvm.dylib' (no such file), '/Users/harvey/Desktop/UCL/Year 4/Modules/HPC/libnvvm.dylib' (no such file), '/usr/local/lib/libnvvm.dylib' (no such file), '/usr/lib/libnvvm.dylib' (no such file)

In [ ]:
visualize(result, npoints)

In [45]:
print(result.shape)

(480000,)
